# Market-basket analysis

In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "mariasilviapancione"
os.environ['KAGGLE_KEY'] = "7990de4ea13c48c128b0617eb347ce5d"
!kaggle datasets download --unzip mohamedbakhet/amazon-books-reviews

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0




  0%|          | 0.00/1.06G [00:00<?, ?B/s]
  0%|          | 1.00M/1.06G [00:00<12:32, 1.51MB/s]
  0%|          | 2.00M/1.06G [00:00<06:39, 2.85MB/s]
  0%|          | 3.00M/1.06G [00:00<04:32, 4.17MB/s]
  0%|          | 5.00M/1.06G [00:01<02:55, 6.48MB/s]
  1%|          | 6.00M/1.06G [00:01<02:42, 6.98MB/s]
  1%|          | 8.00M/1.06G [00:01<02:14, 8.41MB/s]
  1%|          | 9.00M/1.06G [00:01<02:08, 8.82MB/s]
  1%|1         | 11.0M/1.06G [00:01<01:57, 9.60MB/s]
  1%|1         | 13.0M/1.06G [00:01<01:51, 10.1MB/s]
  1%|1         | 15.0M/1.06G [00:02<01:45, 10.6MB/s]
  2%|1         | 17.0M/1.06G [00:02<01:40, 11.1MB/s]
  2%|1         | 19.0M/1.06G [00:02<01:36, 11.6MB/s]
  2%|1         | 21.0M/1.06G [00:02<01:34, 11.8MB/s]
  2%|2         | 23.0M/1.06G [00:02<01:34, 11.9MB/s]
  2%|2         | 25.0M/1.06G [00:02<01:33, 11.9MB/s]
  2%|2         | 27.0M/1.06G [00:03<01:31, 12.1MB/s]
  3%|2         | 29.0M/1.06G [00:03<01:47, 10.3MB/s]
  3%|2         | 31.0M/1.06G [00:03<01:42, 10.8MB/s]
 

In [2]:
import os, sys
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from itertools import combinations
import pandas as pd
from math import ceil
from collections import Counter, defaultdict

## Data pre-processing

In [3]:
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
os.environ["SPARK_LOCAL_IP"] = "127.0.0.1"

spark = (SparkSession.builder
    .master("local[*]")
    .appName("mba")
    .config("spark.driver.host", "127.0.0.1")
    .config("spark.driver.bindAddress", "127.0.0.1")
    .config("spark.pyspark.python", sys.executable)
    .config("spark.pyspark.driver.python", sys.executable)
    .config("spark.network.timeout", "600s")
    .config("spark.executor.heartbeatInterval", "60s")
    .getOrCreate()
)

sc = spark.sparkContext

In [4]:
df = spark.read.csv("Books_rating.csv", header=True, inferSchema=True)
df.show(5)

+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|        Id|               Title|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+----------+--------------------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|1882931173|Its Only Art If I...| NULL| AVCGYZL8FQQTD|"Jim of Oz ""jim-...|               7/7|         4.0|  940636800|Nice collection o...|This is only for ...|
|0826414346|Dr. Seuss: Americ...| NULL|A30TK6U7DNS82R|       Kevin Killian|             10/10|         5.0| 1095724800|   Really Enjoyed It|I don't care much...|
|0826414346|Dr. Seuss: Americ...| NULL|A3UH4UZ4RSVO82|        John Granger|             10/11|         5.0| 1078790400|Essential for eve...|"If people become...|
|0826414346|Dr. Seuss: Ameri

In [5]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)



In [6]:
n_rows = df.count()
print(f"Total number of rows: {n_rows}")

Total number of rows: 3000000


In [7]:
df = df.select("Id", "User_id", "Title")
df.show(5)

+----------+--------------+--------------------+
|        Id|       User_id|               Title|
+----------+--------------+--------------------+
|1882931173| AVCGYZL8FQQTD|Its Only Art If I...|
|0826414346|A30TK6U7DNS82R|Dr. Seuss: Americ...|
|0826414346|A3UH4UZ4RSVO82|Dr. Seuss: Americ...|
|0826414346|A2MVUWT453QH61|Dr. Seuss: Americ...|
|0826414346|A22X4XUPKF66MR|Dr. Seuss: Americ...|
+----------+--------------+--------------------+
only showing top 5 rows


In [8]:
df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+-------+-----+
| Id|User_id|Title|
+---+-------+-----+
|  0| 562250|  208|
+---+-------+-----+



In [9]:
df = df.dropna()

In [10]:
df = df.dropDuplicates(["User_id", "Title"])

In [11]:
n_rows_cleaned  = df.count()
print(f"Total number of rows after cleaning: {n_rows_cleaned}")

Total number of rows after cleaning: 2114559


In [12]:
n_users = df.select("User_id").distinct().count()
n_books = df.select("Id").distinct().count()

print(f"Distinct users: {n_users}")
print(f"Distinct books: {n_books}")

Distinct users: 1008423
Distinct books: 208865


## Dataset subsampling

In [13]:
USE_SUBSAMPLE = True
FRACTION = 0.05

users_df = df.select("User_Id").distinct()

if USE_SUBSAMPLE:
    users_df = users_df.sample(withReplacement=False, fraction=FRACTION, seed=42)

df_sub = df.join(users_df, on="User_Id", how="inner")

In [14]:
n_rows_sub = df_sub.count()
print(f"Number of rows after subsampling: {n_rows_sub}")

Number of rows after subsampling: 108961


In [15]:
n_users_sub = df_sub.select("User_id").distinct().count()
n_books_sub = df_sub.select("Id").distinct().count()

print(f"Number of users after subsampling: {n_users_sub}")
print(f"Number of distinct books after subsampling: {n_books_sub}")

Number of users after subsampling: 50554
Number of distinct books after subsampling: 45460


## Basket construction

In [16]:
MIN_BASKET_SIZE = 2

baskets_df = (
    df_sub
    .groupBy("User_Id")
    .agg(F.collect_set("Id").alias("basket"))
    .filter(F.size("basket") >= MIN_BASKET_SIZE)
)

baskets_rdd = baskets_df.select("basket").rdd.map(lambda r: sorted(r["basket"]))
baskets_rdd.persist()
N = baskets_rdd.count()
print(f"Number of baskets: {N}")

Number of baskets: 14239


## Apriori

 Generate candidate k-itemsets Ck from frequent (k-1) itemsets

In [17]:
def apriori_gen(Lprev, k):

    Lprev_list = sorted(Lprev)
    Lprev_set = set(Lprev)
    Ck = set()

    for i in range(len(Lprev_list)):
        for j in range(i + 1, len(Lprev_list)):

            a = Lprev_list[i]
            b = Lprev_list[j]

            if a[:k-2] == b[:k-2]:

                cand = tuple(sorted(set(a) | set(b)))

                if len(cand) == k:

                    # PRUNE STEP: all (k-1) subsets of cand must be frequent
                    valid = True
                    for sub in combinations(cand, k - 1):
                        if tuple(sorted(sub)) not in Lprev_set:
                            valid = False
                            break

                    if valid:
                        Ck.add(cand)

            else:
                break
                
    return Ck


Count candidate itemsets Ck in the baskets and return those whose support is >= threshold


In [18]:
def count_candidates(baskets_rdd, Ck, support_threshold, sc):

    Ck_bc = sc.broadcast(set(Ck))

    counts = (
        baskets_rdd
        .map(lambda basket: set(basket))
        .flatMap(
            lambda basket: [
                (cand, 1)
                for cand in Ck_bc.value
                if set(cand).issubset(basket)
            ]
        )
        .reduceByKey(lambda a, b: a + b)
    )

    Lk = (
        counts
        .filter(lambda x: x[1] >= support_threshold)
        .collect() 
    )

    Ck_bc.unpersist()
    return dict(Lk)

In [19]:
def apriori_spark(baskets_rdd, support_threshold, sc, max_k=3):

    # PASS 1 — Frequent singletons

    L1_items = (
        baskets_rdd
        .flatMap(lambda basket: [(item, 1) for item in set(basket)]) 
        .reduceByKey(lambda a, b: a + b)
        .filter(lambda x: x[1] >= support_threshold)
        .collect()
    )

    L1_dict = {(item,): cnt for item, cnt in L1_items}
    L1_set = set(item for (item,), _cnt in L1_dict.items())

    frequents = {1: L1_dict}

    if not L1_set:
        return frequents

    Lprev = set(L1_dict.keys())

    # PASSES k = 2 ... max_k

    for k in range(2, max_k + 1):

        if k == 2:
            L1_bc = sc.broadcast(L1_set)

            Lk = (
                baskets_rdd
                .map(lambda basket: [x for x in basket if x in L1_bc.value])
                .filter(lambda basket: len(basket) >= 2)
                .flatMap(lambda basket: [(tuple(sorted(p)), 1) for p in combinations(basket, 2)])
                .reduceByKey(lambda a, b: a + b)
                .filter(lambda x: x[1] >= support_threshold)
                .collect()
            )

            Lk = dict(Lk)

        else:
            Ck = apriori_gen(Lprev, k)
            if not Ck:
                break

            Lk = count_candidates(baskets_rdd, Ck, support_threshold, sc) 

        frequents[k] = Lk
        Lprev = set(Lk.keys())

        if not Lprev:
            break

    return frequents

In [20]:
N = baskets_rdd.count()

support_percentage = 0.003
support_threshold = ceil(support_percentage * N)

print("Support threshold:", support_threshold)

frequent_itemsets = apriori_spark(
    baskets_rdd=baskets_rdd,
    support_threshold=support_threshold,
    sc=sc,
    max_k=3
)

for k in frequent_itemsets:
    print(f"Frequent itemsets of size {k}: {len(frequent_itemsets[k])}")

Support threshold: 43
Frequent itemsets of size 1: 111
Frequent itemsets of size 2: 217
Frequent itemsets of size 3: 317


In [21]:
def show_top(itemsets, k, top_n=20):
    items = sorted(itemsets.get(k, {}).items(), key=lambda x: -x[1])[:top_n]
    for it, cnt in items:
        print(it, cnt)

show_top(frequent_itemsets, 1, 10)
show_top(frequent_itemsets, 2, 10)
show_top(frequent_itemsets, 3, 10)

('B000Q032UY',) 173
('B000ILIJE0',) 173
('B000NWU3I4',) 171
('B000NDSX6C',) 161
('B000GQG5MA',) 158
('B000PCESRE',) 121
('B000PMCF1A',) 119
('B000I3NFKG',) 118
('B000H7EO2G',) 102
('B000J1OR0Y',) 102
('B000ILIJE0', 'B000Q032UY') 173
('B000ILIJE0', 'B000NWU3I4') 171
('B000NWU3I4', 'B000Q032UY') 171
('B000NDSX6C', 'B000Q032UY') 161
('B000ILIJE0', 'B000NDSX6C') 161
('B000NDSX6C', 'B000NWU3I4') 159
('B000GQG5MA', 'B000ILIJE0') 158
('B000GQG5MA', 'B000Q032UY') 158
('B000GQG5MA', 'B000NWU3I4') 156
('B000GQG5MA', 'B000NDSX6C') 152
('B000ILIJE0', 'B000NWU3I4', 'B000Q032UY') 171
('B000ILIJE0', 'B000NDSX6C', 'B000Q032UY') 161
('B000ILIJE0', 'B000NDSX6C', 'B000NWU3I4') 159
('B000NDSX6C', 'B000NWU3I4', 'B000Q032UY') 159
('B000GQG5MA', 'B000ILIJE0', 'B000Q032UY') 158
('B000GQG5MA', 'B000NWU3I4', 'B000Q032UY') 156
('B000GQG5MA', 'B000ILIJE0', 'B000NWU3I4') 156
('B000GQG5MA', 'B000ILIJE0', 'B000NDSX6C') 152
('B000GQG5MA', 'B000NDSX6C', 'B000Q032UY') 152
('B000GQG5MA', 'B000NDSX6C', 'B000NWU3I4') 150


In [22]:
title_df = (df.select("Id", "Title").dropna().dropDuplicates(["Id"]))

In [23]:
bookid_list = [
    "B000ILIJE0",
    "B000NWU3I4",
    "B000Q032UY",
    "B000NDSX6C",
]

In [24]:
title_df \
    .filter(title_df.Id.isin(bookid_list)) \
    .show(truncate=False)

+----------+----------------------------------------------------------------+
|Id        |Title                                                           |
+----------+----------------------------------------------------------------+
|B000ILIJE0|The Hobbit There and Back Again                                 |
|B000NDSX6C|The Hobbit                                                      |
|B000NWU3I4|The Hobbitt, or there and back again; illustrated by the author.|
|B000Q032UY|The Hobbit or There and Back Again                              |
+----------+----------------------------------------------------------------+



In [25]:
support_1 = {k: v / N for k, v in frequent_itemsets[1].items()}

rows = []

for (a, b), cnt_ab in frequent_itemsets[2].items():

    # pairs support
    supp_ab = cnt_ab / N

    # singleton support
    supp_a = support_1[(a,)]
    supp_b = support_1[(b,)]

    # confidence
    conf_a_b = supp_ab / supp_a      
    conf_b_a = supp_ab / supp_b        

    # lift
    lift = supp_ab / (supp_a * supp_b)

    rows.append({
        "item_A": a,
        "item_B": b,
        "count_AB": cnt_ab,
        "support_AB": supp_ab,
        "conf_A_to_B": conf_a_b,
        "conf_B_to_A": conf_b_a,
        "lift": lift
    })

rules_pd = pd.DataFrame(rows)

rules_pd_sorted = rules_pd.sort_values(
    by=["lift", "support_AB"],
    ascending=[False, False]
)

rules_pd_sorted.head(20)

,item_A,item_B,count_AB,support_AB,conf_A_to_B,conf_B_to_A,lift
11,0451519329,1582790485,43,0.003020,1.0,1.000000,331.139535
40,0451519329,B0008588BU,43,0.003020,1.0,1.000000,331.139535
41,0451519329,B000FC1R6O,43,0.003020,1.0,1.000000,331.139535
42,1582790485,158726398X,43,0.003020,1.0,1.000000,331.139535
71,158726398X,B0008588BU,43,0.003020,1.0,1.000000,331.139535
72,158726398X,B000FC1R6O,43,0.003020,1.0,1.000000,331.139535
95,0451519329,158726398X,43,0.003020,1.0,1.000000,331.139535
96,1582790485,B0008588BU,43,0.003020,1.0,1.000000,331.139535
97,1582790485,B000FC1R6O,43,0.003020,1.0,1.000000,331.139535
128,B0008588BU,B000FC1R6O,43,0.003020,1.0,1.000000,331.139535


## SON

In [26]:
def to_itemset(x):
    return tuple(sorted(x))

PASS 1 (Map): Run Apriori locally on one partition to generate candidate itemsets.

In [27]:
def apriori_on_partition(baskets_iter, global_support, N, max_k=3):

    baskets = [set(b) for b in baskets_iter]
    n = len(baskets)
    if n == 0:
        return iter([])

    # local support threshold
    s_local = ceil(global_support * (n / N))

    # k = 1
    c1 = Counter()
    for b in baskets:
        for item in b:
            c1[(item,)] += 1

    L = {iset for iset, cnt in c1.items() if cnt >= s_local}
    all_frequents = set(L)

    # k >= 2
    k = 2
    while L and k <= max_k:
        items = sorted({i for iset in L for i in iset})
        
        Ck = [to_itemset(c) for c in combinations(items, k)]

        L_prev = set(L)
        Ck_pruned = []
        for cand in Ck:
            ok = True
            for sub in combinations(cand, k - 1):
                if to_itemset(sub) not in L_prev:
                    ok = False
                    break
            if ok:
                Ck_pruned.append(cand)

        ck = Counter()
        Ck_sets = [set(c) for c in Ck_pruned]
        for b in baskets:
            if len(b) < k:
                continue
            for cand, cand_set in zip(Ck_pruned, Ck_sets):
                if cand_set.issubset(b):
                    ck[cand] += 1

        L = {iset for iset, cnt in ck.items() if cnt >= s_local}
        all_frequents |= set(L)

        k += 1

    return iter(all_frequents)

PASS 2 (Second Map): Count candidate itemsets on one partition

In [28]:
def count_candidates_on_partition(baskets_iter, candidates_by_k):
    
    baskets = [set(b) for b in baskets_iter]
    if not baskets:
        return iter([])

    counts = Counter()

    for k, cand_list in candidates_by_k.items():
        if not cand_list:
            continue

        cand_sets = [set(c) for c in cand_list]

        for b in baskets:
            if len(b) < k:
                continue
            for cand, cand_set in zip(cand_list, cand_sets):
                if cand_set.issubset(b):
                    counts[cand] += 1

    return iter(counts.items())

In [29]:
def son_frequent_itemsets(baskets_rdd, min_support, max_k=3):

    # First MapReduce
    candidates_rdd = (
        baskets_rdd
        .mapPartitions(lambda it: apriori_on_partition(it, min_support, N, max_k))
        .map(lambda iset: (iset, 1))
        .reduceByKey(lambda a, b: 1)
        .keys()
    )

    candidates = candidates_rdd.collect()

    candidates_by_k = defaultdict(list)
    for c in candidates:
        candidates_by_k[len(c)].append(c)

    bc_cands = sc.broadcast(dict(candidates_by_k))

    # Second MapReduce 2
    freq_rdd = (
        baskets_rdd
        .mapPartitions(lambda it: count_candidates_on_partition(it, bc_cands.value))
        .reduceByKey(lambda a, b: a + b)
        .filter(lambda kv: kv[1] >= min_support)
    )

    return freq_rdd

In [30]:
freq_rdd = son_frequent_itemsets(
    baskets_rdd=baskets_rdd,
    min_support=support_threshold, 
    max_k=3
)

In [31]:
for k in [1, 2, 3]:
    top10 = (freq_rdd
             .filter(lambda kv: len(kv[0]) == k)
             .sortBy(lambda kv: -kv[1])
             .take(10))
    for itemset, cnt in top10:
        print(itemset, cnt)

('B000Q032UY',) 173
('B000ILIJE0',) 173
('B000NWU3I4',) 171
('B000NDSX6C',) 161
('B000GQG5MA',) 158
('B000PCESRE',) 121
('B000PMCF1A',) 119
('B000I3NFKG',) 118
('B000GQK706',) 102
('B000FAIRN2',) 102
('B000ILIJE0', 'B000Q032UY') 173
('B000NWU3I4', 'B000Q032UY') 171
('B000ILIJE0', 'B000NWU3I4') 171
('B000NDSX6C', 'B000Q032UY') 161
('B000ILIJE0', 'B000NDSX6C') 161
('B000NDSX6C', 'B000NWU3I4') 159
('B000GQG5MA', 'B000Q032UY') 158
('B000GQG5MA', 'B000ILIJE0') 158
('B000GQG5MA', 'B000NWU3I4') 156
('B000GQG5MA', 'B000NDSX6C') 152
('B000ILIJE0', 'B000NWU3I4', 'B000Q032UY') 171
('B000ILIJE0', 'B000NDSX6C', 'B000Q032UY') 161
('B000ILIJE0', 'B000NDSX6C', 'B000NWU3I4') 159
('B000NDSX6C', 'B000NWU3I4', 'B000Q032UY') 159
('B000GQG5MA', 'B000ILIJE0', 'B000Q032UY') 158
('B000GQG5MA', 'B000NWU3I4', 'B000Q032UY') 156
('B000GQG5MA', 'B000ILIJE0', 'B000NWU3I4') 156
('B000GQG5MA', 'B000ILIJE0', 'B000NDSX6C') 152
('B000GQG5MA', 'B000NDSX6C', 'B000Q032UY') 152
('B000GQG5MA', 'B000NDSX6C', 'B000NWU3I4') 150


In [32]:
frequent_itemsets_son = defaultdict(dict)

for itemset, cnt in freq_rdd.collect():
    frequent_itemsets_son[len(itemset)][itemset] = cnt

In [33]:
support_1 = {k: v / N for k, v in frequent_itemsets_son[1].items()}

rows = []

for (a, b), cnt_ab in frequent_itemsets_son[2].items():

    # pair support
    supp_ab = cnt_ab / N

    # singleton support
    supp_a = support_1[(a,)]
    supp_b = support_1[(b,)]

    # confidence
    conf_a_b = supp_ab / supp_a
    conf_b_a = supp_ab / supp_b

    # lift
    lift = supp_ab / (supp_a * supp_b)

    rows.append({
        "item_A": a,
        "item_B": b,
        "count_AB": cnt_ab,
        "support_AB": supp_ab,
        "conf_A_to_B": conf_a_b,
        "conf_B_to_A": conf_b_a,
        "lift": lift
    })

rules_pd = pd.DataFrame(rows)

rules_pd_sorted = rules_pd.sort_values(
    by=["lift", "support_AB"],
    ascending=[False, False]
)

rules_pd_sorted.head(20)

,item_A,item_B,count_AB,support_AB,conf_A_to_B,conf_B_to_A,lift
11,0451519329,1582790485,43,0.003020,1.0,1.000000,331.139535
40,0451519329,B000FC1R6O,43,0.003020,1.0,1.000000,331.139535
41,1582790485,158726398X,43,0.003020,1.0,1.000000,331.139535
42,0451519329,B0008588BU,43,0.003020,1.0,1.000000,331.139535
71,158726398X,B000FC1R6O,43,0.003020,1.0,1.000000,331.139535
72,158726398X,B0008588BU,43,0.003020,1.0,1.000000,331.139535
95,0451519329,158726398X,43,0.003020,1.0,1.000000,331.139535
96,1582790485,B000FC1R6O,43,0.003020,1.0,1.000000,331.139535
97,1582790485,B0008588BU,43,0.003020,1.0,1.000000,331.139535
128,B0008588BU,B000FC1R6O,43,0.003020,1.0,1.000000,331.139535
